In [1]:
import pandas as pd
import numpy as np
from scipy import stats

#Data for Campaign 22 (Kitchen Item Recommended)
baseline22 = pd.read_csv(r"C:\Users\level\OneDrive\Documents\Granify Interview Files\HomeCo\campaign_22_baseline.csv")
treatment22 = pd.read_csv(r"C:\Users\level\OneDrive\Documents\Granify Interview Files\HomeCo\campaign_22_treatment.csv")
sessions22 = pd.read_csv(r"C:\Users\level\OneDrive\Documents\Granify Interview Files\HomeCo\campaign_22_sessions.csv")

#Data for Campaign 35 (Furniture Item Recommended) 
baseline35 = pd.read_csv(r"C:\Users\level\OneDrive\Documents\Granify Interview Files\HomeCo\campaign_35_baseline.csv")
treatment35 = pd.read_csv(r"C:\Users\level\OneDrive\Documents\Granify Interview Files\HomeCo\campaign_35_treatment.csv")
sessions35 = pd.read_csv(r"C:\Users\level\OneDrive\Documents\Granify Interview Files\HomeCo\campaign_35_sessions.csv")

#print(baseline22.head())
print(treatment22.head())
#print(sessions22.head())
print(treatment35.head())

   campaignID      group        date  clicked  subtotal
0          22  treatment  2021-01-24        0    121.66
1          22  treatment  2021-01-19        0    119.75
2          22  treatment  2021-01-21        1    113.45
3          22  treatment  2021-01-20        0    111.14
4          22  treatment  2021-01-28        1    122.16
   campaignID      group        date  clicked  subtotal
0          35  treatment  2021-01-17        0     57.12
1          35  treatment  2021-01-06        0     37.25
2          35  treatment  2021-01-05        0     29.09
3          35  treatment  2021-01-23        1    145.21
4          35  treatment  2021-01-07        0    143.16


First, I need to calculate the CTR for the two treatment groups. This is defined as the percentage of the treatment group that 
clicked the recommended item. To calculate this is as simple as summing the "clicked" column (as it is a binary variable) and then dividing the sum by the total number of rows.

In [2]:
#Clickthrough Rate for Campaign 22
CTR_22 = treatment22["clicked"].sum() / len(treatment22.index)

#Clickthrough Rate for Campaign 35
CTR_35 = treatment35["clicked"].sum() / len(treatment35.index)

#"{:%}".format() formats the variables as percentages
print("The CTR for Campaign 22 is", "{:%}".format(CTR_22))
print("The CTR for Campaign 35 is", "{:%}".format(CTR_35))

The CTR for Campaign 22 is 16.465302%
The CTR for Campaign 35 is 15.397665%


We can see that Campaign 22 had a higher CTR than Campaign 35. In fact, the CTR was a little more than one percentage point higher.

Now, I move on to calculate the Average Order Value, or AOV, for the treatment and baseline groups of both campaigns. The AOV is self-explanatory, and so to calculate it I simply calculate the mean of the "subtotal" column of each dataframe.

Then, I will calculate the AOV Lift for each campaign, defined as (AOV treatment / AOV baseline) - 1

In [3]:
#AOV for baseline and treatment groups from Campaign 22
AOV_baseline22 = baseline22["subtotal"].mean()
AOV_treatment22 = treatment22["subtotal"].mean()

#AOV for baseline and treatment groups from Campaign 35
AOV_baseline35 = baseline35["subtotal"].mean()
AOV_treatment35 = treatment35["subtotal"].mean()

print("Campaign 22 baseline group AOV: ", AOV_baseline22)
print("Campaign 22 treatment group AOV:", AOV_treatment22, "\n")
print("Campaign 35 baseline group AOV: ", AOV_baseline35)
print("Campaign 35 treatment group AOV:", AOV_treatment35, "\n")

AOV_lift22 = (AOV_treatment22 / AOV_baseline22) - 1
AOV_lift35 = (AOV_treatment35 / AOV_baseline35) - 1

print("Campaign 22 AOV Lift:", "{:%}".format(AOV_lift22))
print("Campaign 35 AOV Lift:", "{:%}".format(AOV_lift35))

Campaign 22 baseline group AOV:  114.86191877868646
Campaign 22 treatment group AOV: 115.49723714428356 

Campaign 35 baseline group AOV:  114.89859738444568
Campaign 35 treatment group AOV: 116.44605528390429 

Campaign 22 AOV Lift: 0.553115%
Campaign 35 AOV Lift: 1.346803%


But are these differences signficant? One way of assessing this is to run a T-test on the subtotals for the baseline and treatment groups for each campaign.

In [4]:
print("Campaign 22:", stats.ttest_ind(baseline22["subtotal"], treatment22["subtotal"]))
print("Campaign 35:", stats.ttest_ind(baseline35["subtotal"], treatment35["subtotal"]))

Campaign 22: Ttest_indResult(statistic=-5.449007942249054, pvalue=5.1248705327558394e-08)
Campaign 35: Ttest_indResult(statistic=-0.8487191331863405, pvalue=0.39604773671007165)


The p-value for Campaign 22 is extremely small, 5.12e-08, which means the difference in mean AOV between the baseline and treatment group for Campaign 22 is significant. The same cannot be said for differences between the baseline and treatment mean AOVs for Campaign 35, which only had a  p-value of 0.39.

Interestingly, even though the effects of the treatment in Campaign 22 are significant while the effects of the treatment in Campaign 35 are not, the AOV Lift is higher for Campaign 35 compared to Campaign 22. 

I will now merge the campaign data sets with the associated sessions CSV to do the next set of calcuations. To do so, I will concatenate the baseline, treatment, and sessions CSVs together for Campaign 22 and Campaign 33, respectively.

In [5]:
combined22 = pd.concat([baseline22, treatment22, sessions22]).sort_values(by="date")
combined35 = pd.concat([baseline35, treatment35, sessions35]).sort_values(by="date")

Now, I will calculate the CVR lift for both campaigns, defined as (CVR Treatment/CVR Baseline) - 1. To do this, I will first have to calculate the CVR, purchases/count, for the baseline and treatment groups. For this calculation, I am assuming that if the subtotal is > 0 and "click" is *not* 1 (i.e. either 0 or NaN), then one purchase was made, and if the subtotal is > 0 and "click" *is* 1, then two purchases were made (and, of course, if the subtotal is 0 then no purchase was made). I will create a new row that shows the number of purchases for that row.

In [6]:
for i, row in combined22.iterrows():
    
    if row["subtotal"] == 0:
        combined22.at[i, "purchases"] = 0
    
    #elif row["clicked"] == 1:
        #combined22.at[i, "purchases"] = 2
    
    else:
        combined22.at[i, "purchases"] = 1   
        
for i, row in combined35.iterrows():
    
    if row["subtotal"] == 0:
        combined35.at[i, "purchases"] = 0
    
    #elif row["clicked"] == 1:
        #combined35.at[i, "purchases"] = 2
    
    else:
        combined35.at[i, "purchases"] = 1   
    
print(combined35.head())


       campaignID      group        date  clicked  subtotal  purchases
21886          35   baseline  2021-01-01      NaN      0.00        0.0
1552           35  treatment  2021-01-01      0.0    189.92        1.0
4123           35   baseline  2021-01-01      NaN    124.93        1.0
64994          35   baseline  2021-01-01      NaN      0.00        0.0
16644          35  treatment  2021-01-01      NaN      0.00        0.0


In [7]:
CVR_baseline22 = combined22[combined22["group"] == "baseline"]["purchases"].sum() / len(combined22[combined22["group"] == "baseline"])
CVR_treatment22 = combined22[combined22["group"] == "treatment"]["purchases"].sum() / len(combined22[combined22["group"] == "treatment"])

CVR_baseline35 = combined35[combined35["group"] == "baseline"]["purchases"].sum() / len(combined35[combined35["group"] == "baseline"])
CVR_treatment35 = combined35[combined35["group"] == "treatment"]["purchases"].sum() / len(combined35[combined35["group"] == "treatment"])

CVR_lift22 = (CVR_treatment22 / CVR_baseline22) - 1
CVR_lift35 = (CVR_treatment35 / CVR_baseline35) - 1

print("The CVR Lift for Campaign 22 is",  "{:%}".format(CVR_lift22))
print("The CVR Lift for Campaign 35 is",  "{:%}".format(CVR_lift35))

The CVR Lift for Campaign 22 is 0.607488%
The CVR Lift for Campaign 35 is 2.204679%


Now, I will calculate the RPS Lift, defined as (RPS Treatment/RPS Baseline) - 1. To do this, I will first have to calculate the RPS, Revenue/count, for the baseline and treatment groups. The revenue is the value of the "subtotal" column.

In [8]:
RPS_baseline22 = combined22[combined22["group"] == "baseline"]["subtotal"].sum() / len(combined22[combined22["group"] == "baseline"])
RPS_treatment22 = combined22[combined22["group"] == "treatment"]["subtotal"].sum() / len(combined22[combined22["group"] == "treatment"])

RPS_baseline35 = combined35[combined35["group"] == "baseline"]["subtotal"].sum() / len(combined35[combined35["group"] == "baseline"])
RPS_treatment35 = combined35[combined35["group"] == "treatment"]["subtotal"].sum() / len(combined35[combined35["group"] == "treatment"])

RPS_lift22 = (RPS_treatment22 / RPS_baseline22) - 1
RPS_lift35 = (RPS_treatment35 / RPS_baseline35) - 1

print("The RPS Lift for Campaign 22 is",  "{:%}".format(RPS_lift22))
print("The RPS Lift for Campaign 35 is",  "{:%}".format(RPS_lift35))

The RPS Lift for Campaign 22 is 3.112329%
The RPS Lift for Campaign 35 is 2.842794%


Next, I will calculate the daily RPS Lift and CVR lift. I also want to investiage the signficance of the clickthrough rates and the variance in AOV.

In [9]:
import datetime



